# Imports

In [13]:
import calendar
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import tqdm
import imageio.v2 as imageio
import itertools
import matplotlib
import matplotlib.pyplot as plt
import branca.colormap as cm
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [16, 11] 

In [2]:
station_df = pd.read_csv('input_data/all_stations_by_hour_weekday.csv')

In [3]:
station_df = gpd.GeoDataFrame(
    station_df, geometry=gpd.points_from_xy(station_df.coordinatesX, station_df.coordinatesY))
station_df["copy_geometry"] = station_df.geometry


In [4]:
plz_shape_df = gpd.read_file('input_data/plz-5stellig.shp', dtype={'plz': str})
plz_region_df = pd.read_csv(
    'input_data/zuordnung_plz_ort.csv', 
    sep=',', 
    dtype={'plz': str}
)
plz_einwohner_df = pd.read_csv(
    'input_data/plz_einwohner.csv', 
    sep=',', 
    dtype={'plz': str, 'einwohner': int}
)

In [5]:
germany_df = pd.merge(
    left=plz_shape_df, 
    right=plz_region_df, 
    on='plz',
    how='inner'
)


In [6]:
station_df = station_df.set_crs('epsg:4236')

In [7]:
hamburg_df = germany_df.query('ort == "Hamburg"')

joined_df = hamburg_df.sjoin(station_df, how="left")
agg_df = joined_df.groupby(['plz', 'resultHour', 'resultWeekday']).average_res.sum().reset_index()
combined_df = hamburg_df.merge(agg_df, how='left', on="plz")

/Users/felixude/opt/anaconda3/envs/traffic/lib/python3.10/site-packages/geopandas/geodataframe.py:2090: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4236

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


In [63]:
import math
max = math.ceil(unique_coords.average_res.max())
colormap = cm.LinearColormap(colors=['#0d0887', '#cc4778' , '#f0f921',], 
                             index=[0,max/2, max],vmin=0,vmax=max)

In [64]:
colormap

# Creating Station Hour Plots

In [9]:
from tqdm import tqdm
hamburg_df = germany_df.query('ort == "Hamburg"')

station_df.to_csv('station_df.csv', index=False)

for day, hour in tqdm(list(itertools.product(range(1, 8, 1), range(0, 24, 1)))):

    unique_coords = station_df.query(f'resultHour == {hour} and resultWeekday == {day}')
    plt.ion()

    hamburg = hamburg_df.plot(
    )

    cur = unique_coords.plot(ax=hamburg,
                       column='average_res',
                       cmap='Reds', 
                             vmin = 1,
                             vmax = 17,
                             legend_kwds={"label": "Bikes"}


                      )

    cur.set_xlim(9.7, 10.37)
    cur.set_ylim(53.38, 53.75)
    cur.grid(False)
    cur.set_facecolor("lavender")
    w_day = calendar.day_name[day-1]
    if hour < 10:
        hour_string = "0" + str(hour)
    else:
        hour_string = str(hour)
    cur.set_title(f"Hamburg bike stations at {w_day} {hour_string}:00", fontsize=20)
    cur.tick_params(left = False, right = False , labelleft = False ,
            labelbottom = False, bottom = False)
    file_name = f"images/stations_{day}_{hour}.jpg"
    cur.figure.savefig(file_name, dpi=300, bbox_inches='tight')
    plt.close(cur.figure)




100%|██████████████████████████████████████████████| 168/168 [00:54<00:00,  3.08it/s]


In [72]:
unique_coords

,thingID,description,coordinatesX,coordinatesY,resultHour,resultWeekday,average_res,geometry,copy_geometry
38,6,StadtRad-Station S+U Berliner Tor / Beim Stroh...,10.024600,53.553300,23,7,4.131028,POINT (10.02460 53.55330),POINT (10.02460 53.55330)
206,7,StadtRad-Station Alsterschwimmhalle / Ifflands...,10.022148,53.560781,23,7,4.841085,POINT (10.02215 53.56078),POINT (10.02215 53.56078)
374,9,StadtRad-Station Eiffestraße / Normannenweg,10.030364,53.552123,23,7,8.199167,POINT (10.03036 53.55212),POINT (10.03036 53.55212)
542,17,StadtRad-Station Eimsbütteler Straße / Waterlo...,9.953400,53.566200,23,7,4.545729,POINT (9.95340 53.56620),POINT (9.95340 53.56620)
710,19,StadtRad-Station Van-der-Smissen-Straße / Groß...,9.942460,53.543906,23,7,8.269894,POINT (9.94246 53.54391),POINT (9.94246 53.54391)
...,...,...,...,...,...,...,...,...,...
47819,164,StadtRad-Station Enckeplatz / Hütten,9.976950,53.552400,23,7,5.616818,POINT (9.97695 53.55240),POINT (9.97695 53.55240)
47913,194,StadtRad-Station S Königstraße / Struenseestraße,9.942410,53.547100,23,7,2.766374,POINT (9.94241 53.54710),POINT (9.94241 53.54710)
48055,5672,StadtRad-Station Versmannstraße / Gerda-Gmelin...,10.013487,53.539991,23,7,4.559701,POINT (10.01349 53.53999),POINT (10.01349 53.53999)
48221,7780,StadtRad-Station Borsteler Chaussee / Köppenst...,9.978630,53.607440,23,7,5.246032,POINT (9.97863 53.60744),POINT (9.97863 53.60744)


In [30]:
from tqdm import tqdm
for day, hour in tqdm(list(itertools.product(range(1, 8, 1), range(0, 24, 1)))):
    unique_coords = combined_df.query(f'resultHour == {hour} and resultWeekday == {day}')
    fig, ax = plt.subplots()
    plt.ion()
    cur = hamburg_df.plot(ax=ax, color ='grey')
    cur = unique_coords.plot(
                       ax=ax,
                       column='average_res',
                       cmap='Blues', 
                       legend=True,
                       norm=plt.Normalize(vmin=1, vmax=50),
                        legend_kwds={"label": "Bikes"}

                      )


    ax.set_xlim(9.7, 10.37)
    ax.set_ylim(53.38, 53.75)
    ax.grid(False)
    ax.set_facecolor("lavender")
    w_day = calendar.day_name[day-1]
    if hour < 10:
        hour_string = "0" + str(hour)
    else:
        hour_string = str(hour)
    ax.set_title(f"Hamburg bike stations at {w_day} {hour_string}:00")
    cur.tick_params(left = False, right = False , labelleft = False ,
            labelbottom = False, bottom = False)
    file_name = f"images/zip_{day}_{hour}.jpg"
    fig.savefig(file_name, dpi=300, bbox_inches='tight')
    plt.close(fig)



100%|█████████████████████████████████████████| 168/168 [01:39<00:00,  1.69it/s]


# Calculating Closest Station

In [13]:
from shapely.ops import nearest_points
list_of_stations = station_df.geometry.unary_union
def get_closest_station(centroid, list_of_stations):
    
     # find the nearest point and return the corresponding Place value
     nearest = station_df.geometry == nearest_points(centroid, list_of_stations)[1]
     return station_df[nearest].copy_geometry.iloc[0]
    
hamburg_df['Nearest'] = hamburg_df.apply(lambda row: get_closest_station(row.geometry.centroid, list_of_stations),
                                         axis=1)
    

NameError: name 'hamburg_df' is not defined

In [20]:
hamburg_df[["plz", "Nearest"]].head()

KeyError: "['Nearest'] not in index"

In [102]:
import folium
m = folium.Map(location=[53.6, 9.98], zoom_start=10)
m

In [ ]:
import folium
m = folium.Map(location=[53.61, 10.01], zoom_start=14)
for _, r in hamburg_df.loc[hamburg_df["plz"] == "22297"].iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    #folium.Popup(r['BoroName']).add_to(geo_j)
    geo_j.add_to(m)
    
for _, r in joined_df.loc[joined_df["plz"] == "22297"].iterrows():
    lat = r['copy_geometry'].y
    lon = r['copy_geometry'].x
    folium.Marker(location=[lat, lon]).add_to(m)
for _, r in hamburg_df.loc[hamburg_df["plz"] == "22297"].iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    
    lat = r['Nearest'].y
    lon = r['Nearest'].x
    folium.Marker(location=[lat, lon], icon=folium.Icon(color='lightgray', icon='Home', prefix='fa')).add_to(m)
m


In [207]:
import geopy.distance
hamburg_df["centroid"] = hamburg_df.geometry.centroid

hamburg_df["centroid"] = hamburg_df["centroid"].to_crs("EPSG:4490")
hamburg_df["Nearest"] = hamburg_df["Nearest"].set_crs("EPSG:4490")
#hamburg_df["avg_dist"] = hamburg_df["centroid"].distance(hamburg_df["Nearest"])

hamburg_df["avg_dist"] = hamburg_df.apply(lambda row: \
                    geopy.distance.geodesic((row["centroid"].y, row["centroid"].x), 
                    (row["Nearest"].y, row["Nearest"].x)).km, 
                                                                              axis=1)

/tmp/ipykernel_107244/4021066020.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hamburg_df["centroid"] = hamburg_df.geometry.centroid


# Graph Calculation

In [12]:
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
from descartes import PolygonPatch
from IPython.display import IFrame
ox.config(log_console=True, use_cache=True)

/Users/felixude/opt/anaconda3/envs/traffic/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [13]:
bike_station = station_df.loc[station_df["description"].str.contains("Jungfernstieg")].iloc[0]

In [14]:
class DictSmallest(dict):
    def __setitem__(self, key, value):
        if (key not in self) or (key in self and self[key] > value):
            dict.__setitem__(self, key, value)
    def update(self, dict):
        for key, value in dict.items():
            if (key not in self) or (key in self and self[key] > value):
                self[key] =  value
        

In [15]:
unique_station = station_df.groupby('thingID')[["coordinatesY", "coordinatesX"]].min()
unique_station = unique_station.reset_index(drop=True)

all_sub_station = pd.read_csv('input_data/cleaned_stations.csv')

In [16]:
all_sub_station = all_sub_station.rename({"lat": "coordinatesY", "lon": "coordinatesX"}, axis=1)


In [524]:
    trip_times = range(1, 51, 1)

    iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0.3, return_hex=True)
    iso_colors.reverse()

    colormap = cm.LinearColormap(colors=iso_colors)
    colormap = colormap.to_step(index=range(0, 51, 5))
    #colormap.colors.reverse()
    

In [525]:
colormap

In [25]:
%%time

def create_folium_plot(file_name, poi_df, caption, dist=5000):
    # 1 - Create a graph
    G =  ox.graph_from_address("Hamburg, Germany", dist=dist, network_type="walk", simplify=True)
    # 2 - Create nodes geodataframe from Graph network (G)
    gdf_nodes = ox.graph_to_gdfs(G, edges=False)

    list_of_poi = []

    for index, row in poi_df.iterrows():
        list_of_poi.append(ox.distance.nearest_nodes(G, Y=row.coordinatesY, X=row.coordinatesX))

    G = ox.project_graph(G)

    node_distances = DictSmallest()

    for poi in tqdm(list_of_poi):
        tmp_res = nx.shortest_path_length(G, source=poi)
        node_distances.update(tmp_res)
        
    trip_times = range(1, 51, 1)

    iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0.3, return_hex=True)
    iso_colors.reverse()

    colormap = cm.LinearColormap(colors=iso_colors)
    colormap = colormap.to_step(index=range(0, 51, 5))

    colormap.caption = caption




    def color_mapping_function(val):

        for time, color in zip(trip_times, iso_colors):
            if val < time :
                return color

        return iso_colors[-1]

    nc = [color_mapping_function(node_distances[node]) if node in node_distances else 'none' for node in G.nodes()]

    ns = [20 if node in node_distances else 0 for node in G.nodes()]

    coords = {key: {"x": G.nodes[key]["lon"], "y": G.nodes[key]["lat"], "color": color_mapping_function(node_distances[key])} for key in list(G.nodes())}


    import folium
    m = folium.Map(location=[53.555, 9.9914], zoom_start=12,tiles='OpenStreetMap',no_touch=True, prefer_canvas=True, 
                   max_bounds=[53.555, 9.9914])



    for val in coords.values():
        folium.Circle(
          location=[val["y"],val["x"]],
            radius=50,
          #popup="Test",
        stroke=False,
        fill=True,
        color = val["color"],
        fill_opacity=0.3,
        interactive=False

       ).add_to(m)
    colormap.add_to(m)
    m.save(f'maps/{file_name}.html')

CPU times: user 19 µs, sys: 8 µs, total: 27 µs
Wall time: 31 µs


In [27]:
%%time
caption_bike = "Walking distance in minutes to nearest bike station"
create_folium_plot("hamburg_bike_darker", unique_station, dist=15000, caption=caption_bike)

100%|█████████████████████████████████████████████| 288/288 [01:28<00:00,  3.24it/s]


CPU times: user 9min 21s, sys: 2.79 s, total: 9min 24s
Wall time: 9min 49s


In [28]:
%%time
caption_bike = "Walking distance in minutes to nearest subway/-urban station"
create_folium_plot("hamburg_hvv_darker", all_sub_station, dist=15000, caption=caption_bike)

100%|█████████████████████████████████████████████| 285/285 [01:31<00:00,  3.10it/s]


CPU times: user 9min 23s, sys: 1.79 s, total: 9min 24s
Wall time: 9min 24s


In [29]:
all_poi = pd.concat([all_sub_station[["coordinatesY", "coordinatesX"]], unique_station])

In [ ]:
%%time
caption_bike = "Walking distance in minutes to nearest subway/-urban or bike station"
create_folium_plot("hamburg_all_darker", all_poi, dist=15000, caption=caption_bike)